<a href="https://colab.research.google.com/github/ancestor9/Affalatoxin-Analysis/blob/main/LIMS_data_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: /content/drive/MyDrive/식약처/data/250708_통합 LIMS 시험검사 결과(항목 기준)_아플라톡신-20250719T030348Z-1-001/250708_통합 LIMS 시험검사 결과(항목 기준)_아플라톡신 에 있는 데이터 리스트 확인하기

!ls "/content/drive/MyDrive/식약처/data/250708_통합 LIMS 시험검사 결과(항목 기준)_아플라톡신-20250719T030348Z-1-001/250708_통합 LIMS 시험검사 결과(항목 기준)_아플라톡신"

검사명령  모니터링  수입검사  자가품질	품질검사
기타	  수거검사  압몰수품  제조수입


In [2]:
path = '/content/drive/MyDrive/식약처/data/250708_통합 LIMS 시험검사 결과(항목 기준)_아플라톡신-20250719T030348Z-1-001/250708_통합 LIMS 시험검사 결과(항목 기준)_아플라톡신'

In [ ]:
# path 변수 확인
print(f"현재 path 값: {path}")
print(f"path 타입: {type(path)}")

In [ ]:
%pip install xlsxwriter -q

- path 폴더 아래에 있는 데이터를 읽고 파일명, 각 데이터프레임의 고유값, 결측치, 결측치 비율, 대표값 5개 등을 데이터프레임으로 보여주는 함수를 만든 후 각 각의 시트이름을 파일명으로 하는 엑셀파일을 만들어주라

---> claude에게 prompt

### prompt to claude
- 하위폴더에 여러 파일이 있으면 merge하여 하나의 파일로 만든 다음 분석해주고...
- 각 하위폴더에서 읽은 데이터프레임을 사전형으로 만들어 key와 함께 데이터프레임으로 저장해줘

In [14]:
import os
import pandas as pd

def analyze_and_export_dataframes(folder_path):
    """
    하위 폴더별로 파일들을 merge하고 분석해서 엑셀 파일로 저장
    """
    # 폴더 존재 확인
    if not os.path.exists(folder_path):
        print(f"❌ 폴더가 없습니다: {folder_path}")
        return

    # 하위 폴더별 데이터프레임을 저장할 딕셔너리
    folder_dataframes = {}
    analysis_results = {}

    # 하위 폴더 탐색
    for item in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, item)

        if os.path.isdir(subfolder_path):
            print(f"📁 하위폴더 처리 중: {item}")

            # 해당 폴더의 모든 엑셀/CSV 파일 읽기
            folder_files = []

            for filename in os.listdir(subfolder_path):
                if filename.endswith(('.xlsx', '.xls', '.csv')) and not filename.startswith('~$'):
                    file_path = os.path.join(subfolder_path, filename)

                    try:
                        # 파일 읽기
                        if filename.endswith('.csv'):
                            df = pd.read_csv(file_path, encoding='utf-8-sig')
                        else:
                            df = pd.read_excel(file_path)

                        # 파일 출처 정보 추가
                        df['SOURCE_FILE'] = filename
                        folder_files.append(df)

                        print(f"  ✓ {filename} 읽기 완료 ({df.shape[0]}행)")

                    except Exception as e:
                        print(f"  ✗ {filename} 오류: {e}")

            # 해당 폴더의 모든 파일을 merge
            if folder_files:
                if len(folder_files) == 1:
                    merged_df = folder_files[0]
                else:
                    # 같은 컬럼 구조라고 가정하고 concat
                    try:
                        merged_df = pd.concat(folder_files, ignore_index=True)
                        print(f"  🔗 {len(folder_files)}개 파일 merge 완료")
                    except Exception as e:
                        print(f"  ⚠️ Merge 실패, 첫 번째 파일만 사용: {e}")
                        merged_df = folder_files[0]

                # 딕셔너리에 저장
                folder_dataframes[item] = merged_df

                # 분석 수행
                analysis_data = []
                for col in merged_df.columns:
                    try:
                        top5_values = merged_df[col].value_counts().head(5).index.tolist()

                        analysis_data.append({
                            'Column': col,
                            'Unique Count': merged_df[col].nunique(),
                            'Missing Count': merged_df[col].isnull().sum(),
                            'Missing Ratio': f"{(merged_df[col].isnull().sum()/len(merged_df)*100):.2f}%",
                            'Top 5 Values': str(top5_values)
                        })
                    except Exception as e:
                        print(f"    ⚠️ {col} 컬럼 분석 오류: {e}")

                analysis_results[item] = pd.DataFrame(analysis_data)
                print(f"  ✅ {item} 분석 완료 (총 {len(merged_df)}행)")
            else:
                print(f"  ❌ {item} 폴더에 처리할 파일 없음")

    # 결과 저장
    if analysis_results:
        # 1. 분석 결과를 엑셀로 저장
        with pd.ExcelWriter('data_analysis_results.xlsx', engine='openpyxl') as writer:
            for folder_name, analysis_df in analysis_results.items():
                sheet_name = folder_name[:31]  # 시트명 길이 제한
                analysis_df.to_excel(writer, sheet_name=sheet_name, index=False)

        # 2. merge된 데이터프레임들도 별도로 저장
        '''
        with pd.ExcelWriter('merged_dataframes.xlsx', engine='openpyxl') as writer:
            for folder_name, merged_df in folder_dataframes.items():
                sheet_name = folder_name[:31]
                merged_df.to_excel(writer, sheet_name=sheet_name, index=False)
        '''

        print(f"\n🎉 완료!")
        print(f"📊 분석 결과: data_analysis_results.xlsx ({len(analysis_results)}개 시트)")
        print(f"📋 merge된 데이터: merged_dataframes.xlsx ({len(folder_dataframes)}개 시트)")
        print(f"처리된 폴더: {list(folder_dataframes.keys())}")

        # 폴더별 데이터 요약
        for folder_name, df in folder_dataframes.items():
            print(f"  📁 {folder_name}: {df.shape[0]}행 x {df.shape[1]}컬럼")

        return folder_dataframes  # 딕셔너리 형태로 반환

    else:
        print("❌ 분석할 파일이 없습니다.")
        return {}

# 사용법
result_dict = analyze_and_export_dataframes(path)
print(result_dict.keys())  # 딕셔너리 키 확인

📁 하위폴더 처리 중: 자가품질
  ✓ 2021-2022.xlsx 읽기 완료 (32943행)
  ✓ 2017-2018.xlsx 읽기 완료 (23033행)
  ✓ 2019-2020.xlsx 읽기 완료 (28248행)
  ✓ 2023-2025.xlsx 읽기 완료 (43928행)
  ✓ 2015-2016.xlsx 읽기 완료 (6416행)
  🔗 5개 파일 merge 완료
  ✅ 자가품질 분석 완료 (총 134568행)
📁 하위폴더 처리 중: 모니터링
  ✓ 2015-2025.xlsx 읽기 완료 (2475행)
  ✅ 모니터링 분석 완료 (총 2475행)
📁 하위폴더 처리 중: 검사명령
  ✓ 2015-2025.xlsx 읽기 완료 (10행)
  ✅ 검사명령 분석 완료 (총 10행)
📁 하위폴더 처리 중: 수입검사
  ✓ 2015-2016.xlsx 읽기 완료 (3809행)
  ✓ 2017-2018.xlsx 읽기 완료 (12262행)
  ✓ 2019-2021.xlsx 읽기 완료 (62463행)
  ✓ 2024-2025.xlsx 읽기 완료 (40034행)
  ✓ 2022-2023.xlsx 읽기 완료 (50355행)
  🔗 5개 파일 merge 완료
  ✅ 수입검사 분석 완료 (총 168923행)
📁 하위폴더 처리 중: 압몰수품
  ✓ 2015-2025.xlsx 읽기 완료 (69행)
  ✅ 압몰수품 분석 완료 (총 69행)
📁 하위폴더 처리 중: 수거검사
  ✓ 2015-2018.xlsx 읽기 완료 (19472행)
  ✓ 2019-2020.xlsx 읽기 완료 (16957행)
  ✓ 2021-2022.xlsx 읽기 완료 (18071행)
  ✓ 2023-2025.xlsx 읽기 완료 (27289행)
  🔗 4개 파일 merge 완료
  ✅ 수거검사 분석 완료 (총 81789행)
📁 하위폴더 처리 중: 기타
  ✓ 2021-2025.xlsx 읽기 완료 

### 데이터 확인하기

In [17]:
result_dict['수거검사']

,INSPCT_INSTT_CODE,INSPCT_INSTT_NAME,SPLORE_REQEST_NO1,SPLORE_RCEPT_NO,REQEST_DE,JOB_REALM_NAME,INSPCT_PURPS_CODE,INSPCT_KND_CODE,INSPCT_PURPS_NAME,INSPCT_KND_NAME,...,RESULT_VALUE,EXPR_IEM_JDGMNT_NAME,REQEST_INSTT_RCEPT_NO,ORGPLCE_NATION_CODE,MNFCTUR_DE,DISTB_DE,PRLOAD_NM,IMPORT_ENTRPS_NM,ETRYPT_DE,SOURCE_FILE
0,O000093,경상남도보건환경연구원,160100012-1,160100012-001,20160331,식품,IM02000003,IM03000001,수거검사,식품,...,불검출,적합,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-2018.xlsx
1,O000093,경상남도보건환경연구원,160100012-2,160100012-002,20160331,식품,IM02000003,IM03000001,수거검사,식품,...,1.59,적합,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-2018.xlsx
2,O000105,제주도보건환경연구원,160100012-5,160100012-005,20160414,식품,IM02000003,IM03000001,수거검사,식품,...,불검출,상기실험확인함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-2018.xlsx
3,O000105,제주도보건환경연구원,160100012-5,160100012-005,20160414,식품,IM02000003,IM03000001,수거검사,식품,...,불검출,상기실험확인함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-2018.xlsx
4,O000105,제주도보건환경연구원,160100012-8,160100012-008,20160414,식품,IM02000003,IM03000001,수거검사,식품,...,불검출,상기실험확인함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-2018.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81784,O000096,광주광역시보건환경연구원,250700219-1,250700219-001,20250616,축산물,IM02000003,IM03000001,수거검사,식품,...,불검출,적합,2501212830,NaN,2025/04/29 00:00:00.000000,NaN,NaN,NaN,NaN,2023-2025.xlsx
81785,O000096,광주광역시보건환경연구원,250700219-1,250700219-001,20250616,축산물,IM02000003,IM03000001,수거검사,식품,...,불검출,적합,2501212830,NaN,2025/04/29 00:00:00.000000,NaN,NaN,NaN,NaN,2023-2025.xlsx
81786,O000096,광주광역시보건환경연구원,250700220-1,250700220-001,20250616,축산물,IM02000003,IM03000001,수거검사,식품,...,불검출,적합,2501212847,NaN,2025/04/24 00:00:00.000000,NaN,NaN,NaN,NaN,2023-2025.xlsx
81787,O000096,광주광역시보건환경연구원,250700220-1,250700220-001,20250616,축산물,IM02000003,IM03000001,수거검사,식품,...,불검출,적합,2501212847,NaN,2025/04/24 00:00:00.000000,NaN,NaN,NaN,NaN,2023-2025.xlsx


In [15]:
def summarize(df):
    """
    데이터프레임의 각 컬럼에 대한 요약 정보를 담은 데이터프레임을 반환합니다.

    Args:
        df (pd.DataFrame): 요약할 데이터프레임.

    Returns:
        pd.DataFrame: 각 컬럼의 고유값 수, 결측치 수, 결측치 비율, 대표값 5개를 포함하는 요약 데이터프레임.
    """
    summary_list = []
    for col in df.columns:
        unique_count = df[col].nunique()
        missing_count = df[col].isnull().sum()
        missing_ratio = missing_count / len(df) if len(df) > 0 else 0
        top_5_values = df[col].value_counts().nlargest(5).index.tolist()

        summary_list.append({
            'Column': col,
            'Unique Count': unique_count,
            'Missing Count': missing_count,
            'Missing Ratio': f'{missing_ratio:.2%}',
            'Top 5 Values': top_5_values
        })

    summary_df = pd.DataFrame(summary_list)
    return summary_df

In [18]:
summarize(result_dict['수거검사'])

,Column,Unique Count,Missing Count,Missing Ratio,Top 5 Values
0,INSPCT_INSTT_CODE,35,0,0.00%,"[O000100, O000107, O000106, O000105, O000092]"
1,INSPCT_INSTT_NAME,41,0,0.00%,"[서울특별시보건환경연구원, 충청북도보건환경연구원, 충청남도보건환경연구원, 경기도보건..."
2,SPLORE_REQEST_NO1,46430,0,0.00%,"[230102906-1, 240101102-1, 240104019-1, 230100..."
3,SPLORE_RCEPT_NO,46336,0,0.00%,"[230102906-001, 220100396-001, 240101102-001, ..."
4,REQEST_DE,2090,0,0.00%,"[20240520, 20230621, 20161115, 20211126, 20230..."
5,JOB_REALM_NAME,2,0,0.00%,"[식품, 축산물]"
6,INSPCT_PURPS_CODE,1,0,0.00%,[IM02000003]
7,INSPCT_KND_CODE,4,6617,8.09%,"[IM03000001, IM03000016, IM03000002, IM03000003]"
8,INSPCT_PURPS_NAME,1,0,0.00%,[수거검사]
9,INSPCT_KND_NAME,4,6617,8.09%,"[식품, 농산물, 축산물, 수산물]"


In [19]:
# prompt: result_dict를 pickle로 저장해줘

import pickle

# result_dict를 pickle 파일로 저장
with open('result_dict.pkl', 'wb') as f:
    pickle.dump(result_dict, f)

print("result_dict가 result_dict.pkl 파일로 저장되었습니다.")


result_dict가 result_dict.pkl 파일로 저장되었습니다.


In [20]:
# prompt: result_dict.pkl을 읽어줘

# result_dict.pkl 파일 읽기
with open('result_dict.pkl', 'rb') as f:
    loaded_result_dict = pickle.load(f)

print("result_dict.pkl 파일에서 데이터를 성공적으로 읽었습니다.")
print("읽어온 데이터의 키:", loaded_result_dict.keys())

# 읽어온 데이터 확인 (예시: '수거검사' 키의 데이터프레임 확인)
if '수거검사' in loaded_result_dict:
    print("\n'수거검사' 데이터프레임의 head:")
    print(loaded_result_dict['수거검사'].head())
else:
    print("\n읽어온 데이터에 '수거검사' 키가 없습니다.")

result_dict.pkl 파일에서 데이터를 성공적으로 읽었습니다.
읽어온 데이터의 키: dict_keys(['자가품질', '모니터링', '검사명령', '수입검사', '압몰수품', '수거검사', '기타', '제조수입', '품질검사'])

'수거검사' 데이터프레임의 head:
  INSPCT_INSTT_CODE INSPCT_INSTT_NAME SPLORE_REQEST_NO1 SPLORE_RCEPT_NO  \
0           O000093       경상남도보건환경연구원       160100012-1   160100012-001   
1           O000093       경상남도보건환경연구원       160100012-2   160100012-002   
2           O000105        제주도보건환경연구원       160100012-5   160100012-005   
3           O000105        제주도보건환경연구원       160100012-5   160100012-005   
4           O000105        제주도보건환경연구원       160100012-8   160100012-008   

   REQEST_DE JOB_REALM_NAME INSPCT_PURPS_CODE INSPCT_KND_CODE  \
0   20160331             식품        IM02000003      IM03000001   
1   20160331             식품        IM02000003      IM03000001   
2   20160414             식품        IM02000003      IM03000001   
3   20160414             식품        IM02000003      IM03000001   
4   20160414  

### 각 데이터 EDA 수행